In [42]:
import scqubits as scqubits
import numpy as np
import scipy as sci
from system import *

In [43]:
def get_H():
    phi_ex = 0.2
    beta = 1
    Ej = 30.19*2*np.pi
    Ec = 0.1*2*np.pi
    N=1
    #if we want to use scqubits, replace Hs,charge_op with the ones obtained from scqubits
    Hs,charge_op,_ = SNAIL(phi_ex,beta,N,Ej,Ec)
    squid = [Hs,charge_op]
    cavity = scqubits.Oscillator(
        E_osc=5.226,
        truncated_dim = 6
    )
    Hc = np.diag(cavity.eigenvals() - cavity.eigenvals()[0])*2*np.pi
    Vc = cavity.creation_operator() + cavity.annihilation_operator()
    cavity = [Hc,Vc]
    H,H_control = composite_sys(squid,cavity)
    return H,H_control
def get_H_der():
    epsilon = 1e-6
    phi_ex = 0.2 + epsilon
    beta = 1
    Ej = 30.19*2*np.pi
    Ec = 0.1*2*np.pi
    N=1
    #if we want to use scqubits, replace Hs,charge_op with the ones obtained from scqubits
    Hs,charge_op,_ = SNAIL(phi_ex,beta,N,Ej,Ec)
    squid = [Hs,charge_op]
    cavity = scqubits.Oscillator(
        E_osc=5.226,
        truncated_dim = 6
    )
    Hc = np.diag(cavity.eigenvals() - cavity.eigenvals()[0])*2*np.pi
    Vc = cavity.creation_operator() + cavity.annihilation_operator()
    cavity = [Hc,Vc]
    H,H_control = composite_sys(squid,cavity)
    return H,H_control


In [44]:
H, H_control = get_H()
H_der, H_control_der = get_H_der()

In [45]:
np.diag(H)/2/np.pi

array([ 0.        +0.j,  5.226     +0.j, 10.452     +0.j, 15.678     +0.j,
       20.904     +0.j, 26.13      +0.j,  6.14962941+0.j, 11.37562941+0.j,
       16.60162941+0.j, 21.82762941+0.j, 27.05362941+0.j, 32.27962941+0.j,
       12.19537533+0.j, 17.42137533+0.j, 22.64737533+0.j, 27.87337533+0.j,
       33.09937533+0.j, 38.32537533+0.j, 18.13441033+0.j, 23.36041033+0.j,
       28.58641033+0.j, 33.81241033+0.j, 39.03841033+0.j, 44.26441033+0.j,
       23.96406003+0.j, 29.19006003+0.j, 34.41606003+0.j, 39.64206003+0.j,
       44.86806003+0.j, 50.09406003+0.j, 29.6860082 +0.j, 34.9120082 +0.j,
       40.1380082 +0.j, 45.3640082 +0.j, 50.5900082 +0.j, 55.8160082 +0.j,
       35.33356466+0.j, 40.55956466+0.j, 45.78556466+0.j, 51.01156466+0.j,
       56.23756466+0.j, 61.46356466+0.j, 41.07956214+0.j, 46.30556214+0.j,
       51.53156214+0.j, 56.75756214+0.j, 61.98356214+0.j, 67.20956214+0.j,
       47.55319008+0.j, 52.77919008+0.j, 58.00519008+0.j, 63.23119008+0.j,
       68.45719008+0.j, 7

In [62]:
def static_rate(der):
    A = 5e-6
    return np.abs(der*4.4)*A
def calculate_dr(A, omega,H0, Hc,H_der,H_c_der):
    energy01,chi,sk = calculate_floquet_energies(A, omega,H0, Hc)
    epsilon = 1e-6
    energy01_,chi_,sk_ = calculate_floquet_energies(A, omega,H_der, H_c_der)
    der1 = (energy01_ - energy01) / epsilon
    return static_rate(der1),chi,sk
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
def sweep_floquet_energies_derivatives(A, omega_range,H0, Hc,H_der,H_c_der):
    values = Parallel(n_jobs=-1)(delayed(calculate_dr)(A, omegad, H0, Hc,H_der,H_c_der) for omegad in omega_range)
    der_values, chi_values,sk_values = zip(*values)
    return der_values, chi_values,sk_values


In [63]:
static_rate(1)

np.float64(2.2000000000000003e-05)

In [64]:
A = 0.01*2*np.pi
omega_range = np.linspace(6.15,6.2,210)*2*np.pi

In [65]:
result = sweep_floquet_energies_derivatives(A, omega_range,H, H_control,H_der,H_control_der )

In [66]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import pandas as pd

# ...existing code...
data = { 'Frequency (GHz)': omega_range / 2 / np.pi, 'Dephasing Rate (KHz)': result[0], 'Dispersive Shift (GHz)': np.abs(result[1]), 'Self Kerr (GHz)': np.abs(result[2]) }

df = pd.DataFrame(data)
# Create a subplot figure
fig = make_subplots(rows=1, cols=3, subplot_titles=["Dephasing Rate", "Dispersive Shift", "Self Kerr"])

# Add Dephasing Rate trace
fig.add_trace(go.Scatter(x=df["Frequency (GHz)"], y=df["Dephasing Rate (KHz)"], mode="lines", name="Dephasing Rate"),
              row=1, col=1)

# Add Dispersive Shift trace
fig.add_trace(go.Scatter(x=df["Frequency (GHz)"], y=df["Dispersive Shift (GHz)"], mode="lines", name="Dispersive Shift"),
              row=1, col=2)

# Add Self Kerr trace
fig.add_trace(go.Scatter(x=df["Frequency (GHz)"], y=df["Self Kerr (GHz)"], mode="lines", name="Self Kerr"),
              row=1, col=3)

# Use log scale for y-axes
fig.update_yaxes(type="log", row=1, col=1)
fig.update_yaxes(type="log", row=1, col=2)
fig.update_yaxes(type="log", row=1, col=3)

fig.update_layout(title="Three Plots in One Row")
fig.show()

In [67]:
ss_freq = 6.177129*2*np.pi
omega_s = 6.15931123*2*np.pi
detu = ss_freq - omega_s

In [93]:
def static_rate(der):
    A = 5e-6
    return np.abs(der*4.4)*A
def SBw(freq,der):
    A = (5e-6*der)
    print(A)
    return A**2/(np.abs(freq)/2/np.pi)
def shot_noise(me, freq,chi,der):
    gamma = 2e5*2*np.pi*1e-9
    n_bar = 2*me*SBw(freq, der)/2/gamma

    ratio = chi / gamma
    factor = np.real(np.sqrt(((1 + 1j*ratio)**2 + 4*1j*ratio*n_bar)) - 1) / 2
    return (gamma * factor)
def coherent_noise(me, freq,chi):
    chi = chi*1e9
    freq = freq*1e9
    gamma = 2e4
    return 1/2*(me*chi**2*gamma/ ((chi/2)**2+(gamma/2)**2+(freq)**2))

In [94]:
evals,_ = np.linalg.eigh(H)
evals = evals - evals[0]
evals_der,_ = np.linalg.eigh(H_der)
evals_der = evals_der - evals_der[0]

In [95]:
der = ((evals_der - evals)/1e-6)[2]
der

np.float64(-44.42134758164684)

In [96]:
1/static_rate(der)

np.float64(1023.2590393841514)

In [97]:
detu/2/np.pi

0.01781777000000003

In [99]:
1/SBw(detu,der)

-0.00022210673790823424


np.float64(361185.1459251213)

In [100]:
1/shot_noise((A/2/detu)**2, detu,0.002*2*np.pi,der)

-0.00022210673790823424


np.float64(4632550.392711803)